# Vision Capital Growth Multifactor ETF Trading Model
MN30470 Investment & Trading


This Notebook provides the full algorithm used by Vision Capital as a systematic tool to investing in the Market Class Global Index. The output indicates how many shares should be held per stock taking into account the exposure of the Fund in the MCGI itself to reach exposures greater than 10%.

In [1]:
#On 01/03/2020 by Victor Alabrune <va313@bath.ac.uk>
import numpy as np
import pandas as pd
import math
from pandas import ExcelWriter
import pickle
from sklearn.linear_model import LinearRegression

In [2]:
#Year
y = 2028

#Total Balance
T_Balance = 178415125

#Style Factor Tilts
Momentum_Tilt = -0.0455
Growth_Tilt = 0.382
Quality_Tilt = 0.13
Value_Tilt = -0.194
Size_Tilt = -0.03
LowVolatility_Tilt = -0.015
Carry_Tilt = -0.18

#Cash
Cash = 0.03

In [3]:
#Upload Excel Files
i1 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-AITE-2028.xlsx')
#i2 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2023/MC-Company-BIOU-2023.xlsx')
i2 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-BRUE-2028.xlsx')
i3 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-BZME-2028.xlsx')
i4 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-CACG-2028.xlsx')
i5 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-COTU-2028.xlsx')
i6 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-DALG-2028.xlsx')
i7 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-HKBE-2028.xlsx')
i8 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-ICOU-2028.xlsx')
i9 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-ITKJ-2028.xlsx')
i10 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-IXXU-2028.xlsx')
i11 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-JNJU-2028.xlsx')
i12 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-KWIJ-2028.xlsx')
i13 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-MAXE-2028.xlsx')
i14 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-NEOE-2028.xlsx')
i15 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-NSCJ-2028.xlsx')
i16 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-NSGG-2028.xlsx')
i17 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-OAKG-2028.xlsx')
i18 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-SOJU-2028.xlsx')
i19 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-SOLJ-2028.xlsx')
i20 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-UPCU-2028.xlsx')
i21 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-XCLE-2028.xlsx')
i22 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-XSIJ-2028.xlsx')
i23 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-ZACU-2028.xlsx')
i24 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Company-ZENG-2028.xlsx')
i25 = pd.read_excel('/Users/victoralabrune/Desktop/Market Class/2028/MC-Research-2028.xlsx')

#Create a list to run Company files loop
df_list = [i1,i2,i3,i4,i5,i6,i7,i8,i9,i10,i11,i12,i13,i14,i15,i16,i17,i18,i19,i20,i21,i22,i23,i24]

In [4]:
#Row calculations for df.iat
Year0 = y - 2014
Year1 = y - 2013
Year2 = y - 2012
Year3 = y - 2011
Year4 = y - 2010
Year5 = y - 2009

In [5]:
#Run through calculations from Company files
for i, df in enumerate(df_list, 1):
    
    #Create array for volatility calculation
    p0 = df.iat[19,Year0]
    p1 = df.iat[19,Year1]
    p2 = df.iat[19,Year2]
    p3 = df.iat[19,Year3]
    p4 = df.iat[19,Year4]
    p5 = df.iat[19,Year5]
    Price = np.array([p1,p2,p3,p4,p5])
    
    #Create array for volatility calculation
    eps0 = df.iat[31,Year0]
    eps1 = df.iat[31,Year1]
    eps2 = df.iat[31,Year2]
    eps3 = df.iat[31,Year3]
    eps4 = df.iat[31,Year4]
    eps5 = df.iat[31,Year5]
    EPS = np.array([eps1,eps2,eps3,eps4,eps5])
    
    #Create array for volatility calculation
    sales0 = df.iat[26,Year0]
    sales1 = df.iat[26,Year1]
    sales2 = df.iat[26,Year2]
    sales3 = df.iat[26,Year3]
    sales4 = df.iat[26,Year4]
    sales5 = df.iat[26,Year5]
    Sales = np.array([sales1,sales2,sales3,sales4,sales5])
    
    #Create array for regression calculation
    year0 = df.iat[18,Year0]
    year1 = df.iat[18,Year1]
    year2 = df.iat[18,Year2]
    year3 = df.iat[18,Year3]
    year4 = df.iat[18,Year4]
    year5 = df.iat[18,Year5]
    Year = np.array([year1,year2,year3,year4,year5]).reshape((-1, 1))
    
    Momentum = (((p5 - p4)/p4)*0.4 + ((p4 - p3)/p3)*0.3 + ((p3 - p2)/p2)*0.2 + ((p2 - p1)/p1)*0.1) #Weighted Momentum
    Volatility = np.std(Price) #5yr stock volatility
    EPS_Growth = ((eps5 - eps0)/eps0)/np.mean(EPS) #5yr EPS growth
    EPS_Variability = np.std(EPS)/np.mean(EPS) #5yr EPS variability
    Sales_Growth = ((sales5 - sales0)/sales0)/np.mean(Sales) #5yr Sales growth
    Sales_Variability = np.std(Sales)/np.mean(Sales) #5yr Sales variability
    Net_Profit_Margin = df.iat[30,Year5]/df.iat[26,Year5] #Net Income/Total Revenue
    
    Average_Shareholder_Equity = df.iat[43,Year5]*1000/df.iat[47,Year5]
    ROE = df.iat[30,Year5]/Average_Shareholder_Equity
    
    ROA = df.iat[30,Year5]/df.iat[39,Year5] #Net Income/Total Assets
    Debt_Assets = df.iat[42,Year5]/df.iat[39,Year5] #Total Liabilities/Total Assets
    CROCI = df.iat[28,Year5]/df.iat[43,Year5] #EBITDA/Total Shareholder's Equity
    Enterprise_Multiple = df.iat[28,Year5]/((df.iat[19,Year5]*df.iat[47,Year5])+df.iat[42,Year5]-df.iat[35,Year5])
    Annual_TradedVal = df.iat[22,Year5]/df.iat[47,Year5] #Traded Volume/num. of shares
    Cash_Earnings_to_Price = df.iat[28,Year5]/df.iat[19,Year5] #EBITDA/Price
    Asset_Turnover = df.iat[26,Year5]/df.iat[39,Year5] #Sales/Total Assets 
    
    #Sales regression against Year
    model = LinearRegression()
    model.fit(Year, Sales)
    model = LinearRegression().fit(Year, Sales)
    r_sq = model.score(Year, Sales)
    Sales_GrowthR = float(model.coef_)
    
    #Create DataFrame df_calc to record calculations
    Name = df.columns[0]
    df_calc = pd.DataFrame()
    df_calc['Name'] = [Name]
    df_calc['Stock Price'] = p5
    df_calc['Momentum'] = [Momentum]
    df_calc['NPM'] = [Net_Profit_Margin]
    df_calc['High Vol'] = [Volatility]
    df_calc['EPS Var'] = [EPS_Variability]
    df_calc['EPS Growth'] = [EPS_Growth]
    df_calc['ROE'] = [ROE]
    df_calc['DA'] = [Debt_Assets]
    df_calc['CROCI'] = [CROCI]
    df_calc['EM'] = [Enterprise_Multiple]
    df_calc['Annual Traded Value'] = [Annual_TradedVal]
    df_calc['Sales Var'] = [Sales_Variability]
    df_calc['Cash Earnings to Price'] = [Cash_Earnings_to_Price]
    df_calc['Sales Growth'] = [Sales_Growth]
    df_calc['Asset T'] = [Asset_Turnover]
    df_calc['ROA'] = [ROA]
    df_calc['Sales_GR'] = [Sales_GrowthR]
    
    #DataFrame serialization 
    df_calc.to_pickle(Name)
    
#Create DataFrame df_main to combine all df_calc DataFrames (ALPHABETICAL ORDERING NEEDED)
df_main = pd.concat([pd.read_pickle('Auto Italia (AITE)'),
                     pd.read_pickle('Benzin Motors (BZME)'),
                     pd.read_pickle('Bruer AG (BRUE)'),
                     pd.read_pickle('Canterbury Carbonates (CACG)'),
                     pd.read_pickle('Colorado Technology (COTU)'),
                     pd.read_pickle('Dallagio PLC (DALG)'),
                     pd.read_pickle('Henson-Kitt Beverages (HKBE)'),
                     pd.read_pickle('Icarus Oil (ICOU)'),
                     pd.read_pickle('Infinity Tek (ITKJ)'),
                     pd.read_pickle('Ixxon Inc (IXXU)'),
                     pd.read_pickle('Janssen & Janssen (JNJU)'),
                     pd.read_pickle('Kiwi Brewery Inc (KWIJ)'),
                     pd.read_pickle('MAXI SA (MAXE)'),
                     pd.read_pickle('Nassan Cars (NSCJ)'),
                     pd.read_pickle('NeoPharma SpA (NEOE)'),
                     pd.read_pickle('North Shore Gas (NSGG)'),
                     pd.read_pickle('Oakhouse Electronics (OAKG)'),
                     pd.read_pickle('SodaJohnson (SOJU)'),
                     pd.read_pickle('Solaris Energy (SOLJ)'),
                     pd.read_pickle('Upel Corp (UPCU)'),
                     pd.read_pickle('X-Cell AG (XCLE)'),
                     pd.read_pickle('XSI Electronics (XSIJ)'),
                     pd.read_pickle('ZACCAR Inc (ZACU)'),
                     pd.read_pickle('Zenith PLC (ZENG)')])

In [6]:
#Run through calculations from Research file
df_research = i25.drop([0,1,2,3,4,5,6,12,13,14,22,23,29,30]) #Remove irrelevant rows
df_research = df_research.sort_values(by='Index') #Sort alphabetically to match df_main
df_research['Market Cap'] = df_research['Market Cap\n(mm)']/df_research['Market Cap\n(mm)'].sum() #Market Cap


In [7]:
#Transfer df_research values to df_main
df_main['Market Cap'] = df_research['Market Cap'].values
df_main['P/E Ratio'] = df_research['P/E Ratio'].values
df_main['Div\Y'] = df_research['Dividend\nYield'].values
df_main['B/P'] = 1/df_research['Price/\nBook'].values
df_main['EY'] = 1/df_research['P/E Ratio'].values
df_main['Cash E/E'] = df_main['Cash Earnings to Price'] - df_main['EY']
df_main['Dividend'] = df_research['Dividend'].values 
df_main['EPS'] = df_research['EPS'].values
df_main['Retention Rate'] = 1 - (df_main['Dividend'] / df_main['EPS'])
df_main['SGR'] = df_main['Retention Rate'] * df_main['ROE']


In [8]:
#Standardize df_main, z=(x-μ)/σ
df_main['Momentum (z)'] = (df_main['Momentum']-df_main['Momentum'].mean())/df_main['Momentum'].std()
df_main['Low Volatility (z)'] = -((df_main['High Vol']-df_main['High Vol'].mean())/df_main['High Vol'].std())
df_main['EPS Variability (z)'] = - ((df_main['EPS Var'] - df_main['EPS Var'].mean())/df_main['EPS Var'].std())
df_main['ROE (z)'] = (df_main['ROE'] - df_main['ROE'].mean())/df_main['ROE'].std()
df_main['Debt to Assets (z)'] = - ((df_main['DA'] - df_main['DA'].mean())/df_main['DA'].std())
df_main['CROCI (z)'] = (df_main['CROCI'] - df_main['CROCI'].mean())/df_main['CROCI'].std()
df_main['Market Cap (z)'] = (df_main['Market Cap'] - df_main['Market Cap'].mean())/df_main['Market Cap'].std()
df_main['P/E Ratio (z)'] = (df_main['P/E Ratio'] - df_main['P/E Ratio'].mean())/df_main['P/E Ratio'].std()
df_main['Dividend\nYield (z)'] = (df_main['Div\Y'] - df_main['Div\Y'].mean())/df_main['Div\Y'].std()
df_main['Book to Price (z)'] = (df_main['B/P'] - df_main['B/P'].mean())/df_main['B/P'].std()
df_main['Net Profit Margin (z)'] = (df_main['NPM'] - df_main['NPM'].mean())/df_main['NPM'].std()
df_main['Earnings Yield (z)'] = (df_main['EY'] - df_main['EY'].mean())/df_main['EY'].std()
df_main['Enterprise Multiple (z)'] = (df_main['EM'] - df_main['EM'].mean())/df_main['EM'].std()
df_main['Sales Var (z)'] = - ((df_main['Sales Var'] - df_main['Sales Var'].mean())/df_main['Sales Var'].std())
df_main['Cash Earnings to Earnings (z)'] = (df_main['Cash E/E']-df_main['Cash E/E'].mean())/df_main['Cash E/E'].std()
df_main['EPS Growth (z)'] = (df_main['EPS Growth'] - df_main['EPS Growth'].mean())/df_main['EPS Growth'].std()
df_main['Asset Turnover (z)'] = (df_main['Asset T'] - df_main['Asset T'].mean())/df_main['Asset T'].std()
df_main['ROA (z)'] = (df_main['ROA'] - df_main['ROA'].mean())/df_main['ROA'].std()
df_main['Sales Growth R (z)'] = (df_main['Sales_GR'] - df_main['Sales_GR'].mean())/df_main['Sales_GR'].std()
df_main['Sustainable Growth Rate (z)'] = (df_main['SGR'] - df_main['SGR'].mean())/df_main['SGR'].std()

#Create Factor Sleeves
df_main['Quality (z)'] = (df_main['Sales Var (z)'] + 
                          df_main['EPS Variability (z)'] + 
                          df_main['CROCI (z)'] + 
                          df_main['Net Profit Margin (z)'] + 
                          df_main['ROA (z)'] + 
                          df_main['Debt to Assets (z)'])/6

df_main['Value (z)'] = (df_main['Book to Price (z)'] + 
                        df_main['Earnings Yield (z)'] + 
                        df_main['Enterprise Multiple (z)'])/3

df_main['Growth (z)'] = (0.25* df_main['Sustainable Growth Rate (z)'] + 
                         0.5*df_main['Sales Growth R (z)'] + 
                         0.25*df_main['Asset Turnover (z)'])

In [9]:
#Create Master DataFrame
df_master = df_main.filter(['Name'])
df_master['Market Cap'] = df_main['Market Cap']
df_master['Momentum'] = df_main['Momentum (z)'].values
df_master['Quality'] = df_main['Quality (z)'].values
df_master['Value'] = df_main['Value (z)'].values
df_master['Large Size'] = df_main['Market Cap (z)'].values
df_master['Low Volatility'] = df_main['Low Volatility (z)'].values
df_master['Carry'] = df_main['Dividend\nYield (z)'].values
df_master['Growth'] = df_main['Growth (z)'].values

#Winsorize df_master to trim outliers -3<z<3
df_master_W = df_main.filter(['Name'])
df_master_W['Stock Price'] = df_main['Stock Price']
df_master_W['Market Cap'] = df_main['Market Cap']
df_master_W['Momentum'] = np.where(df_master['Momentum']>3, 3, df_master['Momentum'])
df_master_W['Momentum'] = np.where(df_master['Momentum']<-3, -3, df_master['Momentum'])
df_master_W['Quality'] = np.where(df_master['Quality']>3, 3, df_master['Quality'])
df_master_W['Quality'] = np.where(df_master['Quality']<-3, -3, df_master['Quality'])
df_master_W['Value'] = np.where(df_master['Value']>3, 3, df_master['Value'])
df_master_W['Value'] = np.where(df_master['Value']<-3, -3, df_master['Value'])
df_master_W['Large Size'] = np.where(df_master['Large Size']>3, 3, df_master['Large Size'])
df_master_W['Large Size'] = np.where(df_master['Large Size']<-3, -3, df_master['Large Size'])
df_master_W['Low Volatility'] = np.where(df_master['Low Volatility']>3, 3, df_master['Low Volatility'])
df_master_W['Low Volatility'] = np.where(df_master['Low Volatility']<-3, -3, df_master['Low Volatility'])
df_master_W['Carry'] = np.where(df_master['Carry']>3, 3, df_master['Carry'])
df_master_W['Carry'] = np.where(df_master['Carry']<-3, -3, df_master['Carry'])
df_master_W['Growth'] = np.where(df_master['Growth']>3, 3, df_master['Growth'])
df_master_W['Growth'] = np.where(df_master['Growth']<-3, -3, df_master['Growth'])

#Weighted average z-score with style tilt
df_master_W['New Z-Score'] = ((df_master_W['Growth']*Growth_Tilt)+
                            (df_master_W['Large Size']*Size_Tilt)+
                            (df_master_W['Momentum']*Momentum_Tilt)+
                            (df_master_W['Low Volatility']*LowVolatility_Tilt)+
                            (df_master_W['Quality']*Quality_Tilt)+
                            (df_master_W['Value']*Value_Tilt)+
                            (df_master_W['Carry']*Carry_Tilt))

#Exponential to bring all values to positive
df_master_W['Tilt'] = df_master_W['Market Cap']*(2.718281**(df_master_W['New Z-Score']))

#Convert to percentage for stock weight allocation
df_master_W['Total Stock Exposure (%)'] = ((df_master_W['Tilt']/df_master_W['Tilt'].sum())*100)*(1-Cash)
df_master_W['25% MCGI (%)'] = (df_master_W['Market Cap']/4)*100
df_master_W['Weight Allocation (%)'] = df_master_W['Total Stock Exposure (%)'] - df_master_W['25% MCGI (%)']
df_master_W['# of Stocks to Hold'] = ((df_master_W['Weight Allocation (%)']/100)*T_Balance)/df_master_W['Stock Price']

df_master_W['CHECK'] = (df_master_W['Total Stock Exposure (%)'] - 
                        (df_master_W['25% MCGI (%)'] + 
                         df_master_W['Weight Allocation (%)']))

df_master_W.loc[df_master_W['Weight Allocation (%)'] <= 10, '10% CAP CHECK'] = 'OK'
df_master_W.loc[df_master_W['Weight Allocation (%)'] > 10, '10% CAP CHECK'] = 'ERROR'

In [10]:
#Final DataFrame and Checks
print('100 - Cash =', df_master_W['Total Stock Exposure (%)'].sum())
print('Cash:', Cash*100)
df_master_W.set_index('Name').iloc[:,11:15]

100 - Cash = 96.99999999999999
Cash: 3.0


,Total Stock Exposure (%),25% MCGI (%),Weight Allocation (%),# of Stocks to Hold
Name,,,,
Auto Italia (AITE),1.12499,0.320691,0.804296,87243.8
Benzin Motors (BZME),3.06588,1.08135,1.98453,55109
Bruer AG (BRUE),4.70943,1.15859,3.55084,68348.7
Canterbury Carbonates (CACG),1.66823,0.385109,1.28312,39152.5
Colorado Technology (COTU),3.12432,0.656958,2.46736,134684
Dallagio PLC (DALG),3.73939,0.869421,2.86997,252925
Henson-Kitt Beverages (HKBE),0.838274,0.214106,0.624168,20840.1
Icarus Oil (ICOU),1.48517,0.378381,1.10679,59835.1
Infinity Tek (ITKJ),1.09938,0.23701,0.862368,46181.8
